In [1]:
import numpy as np
import pandas as pd

# -------------------
# PARAMETERS
# -------------------
baseline_year = 2015
sim_years = 10
n_clusters = 132
n_diagnoses = 131

years = list(range(baseline_year, baseline_year + sim_years + 1))

# -------------------
# DUMMY INPUT DATA
# -------------------
df_birth = pd.DataFrame({
    0: [1000] * sim_years,
    1: [950] * sim_years
}, index=range(baseline_year, baseline_year + sim_years))

age_groups_fy = [(0, 4), (5, 9), (10, 14), (15, 19), (20, 24),
                 (25, 29), (30, 34), (35, 39), (40, 44), (45, 49)]
df_migr_female = pd.DataFrame(0, index=range(len(age_groups_fy)),
                               columns=[str(y) for y in range(baseline_year, baseline_year + sim_years)])
df_migr_male = df_migr_female.copy()

tm = [pd.DataFrame(np.eye(n_clusters)) for _ in age_groups_fy]
tm_m = [pd.DataFrame(np.eye(n_clusters)) for _ in age_groups_fy]

new_mp = pd.DataFrame(np.zeros((n_clusters, n_diagnoses)))

diag_index = ['A00-B99', 'A15-A19', 'B20-B24', 'B15-B19, B94.2', 'C00-C97', 'D00-D48', 'D50-D89', 'E00-E90', 'E10-E14', 'F00-F99',
              'F10-F19', 'G00-H95', 'I00-I99', 'I20-I25', 'I60-I69', 'J00-J99', 'J09-J18', 'J40-J47', 'K00-K93', 'L00-L99',
              'M00-M99', 'N00-N99', 'N00-N29']
diag_nums = [[0] + list(range(2, 11)) + list(range(13, 18)) + [19] + [20], [1] + [18], [12], [11], [21], list(range(22, 25)),
             list(range(25, 31)), [31] + list(range(33, 39)), [32], [39] + list(range(41, 49)), [40], list(range(50, 76)),
             list(range(76, 79)) + list(range(83, 86)) + [80] + [81], [79], [82],
             list(range(91, 96)) + [86] + [88] + [89] + list(range(91, 96)), [87], [90],
             [96] + list(range(98, 103)) + list(range(104, 106)) + [97] + [103],
             list(range(106, 114)), list(range(114, 120)), list(range(125, 131)), list(range(120, 125))]

n_age_cols = 10
def create_dummy_death_df():
    df = pd.DataFrame(
        np.random.randint(0, 800, size=(len(diag_index), n_age_cols)),
        index=diag_index,
        columns=range(1, n_age_cols + 1)
    )
    df['i'] = range(len(diag_index))
    return df

df_death_numbers_w = [create_dummy_death_df() for _ in years]
df_death_numbers_m = [create_dummy_death_df() for _ in years]

df_death_indep_w = pd.DataFrame(0, index=range(baseline_year, baseline_year + sim_years), columns=range(len(age_groups_fy)))
df_death_indep_m = df_death_indep_w.copy()

# -------------------
# INITIAL POPULATION
# -------------------
n_initial = 5000
age = np.random.randint(0, 90, size=n_initial)
sex = np.random.choice([1, 2], size=n_initial)  # 1 = female, 2 = male
deathstate = np.zeros(n_initial, dtype=int)
diagnoses = np.zeros((n_initial, n_diagnoses), dtype=int)
cluster = np.zeros((n_initial, sim_years + 1), dtype=int)
migr = np.zeros(n_initial, dtype=int)

# -------------------
# FUNCTIONS
# -------------------
def migration(age, sex, deathstate, diagnoses, cluster, df_migr_female, df_migr_male,
              year_as_str, year, age_groups_fy, sim_years, migr, starting_year):
    for a, (al, au) in enumerate(age_groups_fy):
        nw = int(np.round(df_migr_female[year_as_str].iloc[a]))
        nm = int(np.round(df_migr_male[year_as_str].iloc[a]))

        if nw > 0:
            mask_f = (sex == 1) & (al <= age) & (age <= au) & (deathstate == 0)
            c_cl = np.random.choice(cluster[mask_f][:, year], size=nw, replace=False) if np.any(mask_f) else np.zeros(nw)
            new_p = np.zeros((nw, sim_years + 1))
            new_p[:, year] = c_cl
            cluster = np.vstack([cluster, new_p])
            age = np.append(age, al + np.random.choice(range(5), size=nw))
            sex = np.append(sex, [1] * nw)
            deathstate = np.append(deathstate, [0] * nw)
            diagnoses = np.vstack([diagnoses, np.zeros((nw, n_diagnoses))])
            migr = np.append(migr, [starting_year] * nw)

        elif nw < 0:
            mask_f = (sex == 1) & (al <= age) & (age <= au) & (deathstate == 0)
            if np.any(mask_f):
                rand_ind = np.random.choice(np.where(mask_f)[0], size=abs(nw), replace=False)
                deathstate[rand_ind] += 3

        if nm > 0:
            mask_m = (sex == 2) & (al <= age) & (age <= au) & (deathstate == 0)
            c_cl = np.random.choice(cluster[mask_m][:, year], size=nm, replace=False) if np.any(mask_m) else np.zeros(nm)
            new_p = np.zeros((nm, sim_years + 1))
            new_p[:, year] = c_cl
            cluster = np.vstack([cluster, new_p])
            age = np.append(age, al + np.random.choice(range(5), size=nm))
            sex = np.append(sex, [2] * nm)
            deathstate = np.append(deathstate, [0] * nm)
            diagnoses = np.vstack([diagnoses, np.zeros((nm, n_diagnoses))])
            migr = np.append(migr, [starting_year] * nm)

        elif nm < 0:
            mask_m = (sex == 2) & (al <= age) & (age <= au) & (deathstate == 0)
            if np.any(mask_m):
                rand_ind = np.random.choice(np.where(mask_m)[0], size=abs(nm), replace=False)
                deathstate[rand_ind] += 3

    return age, sex, deathstate, diagnoses, cluster, migr

def dying_indep(age, sex, deathstate, deaths_indep_female, deaths_indep_male, year, age_groups):
    for idx, (age_low, age_up) in enumerate(age_groups):
        n_female = int(deaths_indep_female.loc[int(year)][idx])
        n_male = int(deaths_indep_male.loc[int(year)][idx])
        mask_f = (sex == 1) & (age_low <= age) & (age <= age_up) & (deathstate == 0)
        mask_m = (sex == 2) & (age_low <= age) & (age <= age_up) & (deathstate == 0)
        if n_female > 0 and np.any(mask_f):
            rand_f = np.random.choice(np.where(mask_f)[0], min(n_female, np.sum(mask_f)), replace=False)
            deathstate[rand_f] += year + 100
        if n_male > 0 and np.any(mask_m):
            rand_m = np.random.choice(np.where(mask_m)[0], min(n_male, np.sum(mask_m)), replace=False)
            deathstate[rand_m] += year + 100
    return deathstate

def dying(age, sex, deathstate, diagnoses, deaths_female, deaths_male, year, age_groups, diag_groups, diag_nums):
    count_female = pd.DataFrame(0, index=diag_groups, columns=range(1, len(age_groups) + 1))
    count_male = pd.DataFrame(0, index=diag_groups, columns=range(1, len(age_groups) + 1))
    count_female['i'] = range(len(diag_groups))
    count_male['i'] = range(len(diag_groups))

    for a in range(1, 11):
        age_low, age_up = (a * 10) - 10, (a * 10) - 1
        if a == 10:
            age_up = 150
        for i in range(len(deaths_female)):
            mask_f = (sex == 1) & (age_low <= age) & (age <= age_up) & (deathstate == 0) & (np.any(diagnoses[:, diag_nums[i]], axis=1))
            mask_m = (sex == 2) & (age_low <= age) & (age <= age_up) & (deathstate == 0) & (np.any(diagnoses[:, diag_nums[i]], axis=1))
            count_female.loc[diag_groups[i], a] += np.count_nonzero(mask_f)
            count_male.loc[diag_groups[i], a] += np.count_nonzero(mask_m)

    dr_eff_female = deaths_female / count_female
    dr_eff_male = deaths_male / count_male

    for a in range(1, 11):
        age_low, age_up = (a * 10) - 10, (a * 10) - 1
        if a == 10:
            age_up = 150
        sorted_female = deaths_female.sort_values(by=[a], ascending=False)
        sorted_male = deaths_male.sort_values(by=[a], ascending=False)

        for i in range(len(sorted_female)):
            d_id_f = int(sorted_female['i'].iloc[i])
            d_id_m = int(sorted_male['i'].iloc[i])
            n_f = int(np.round(sorted_female[a].iloc[i]))
            n_m = int(np.round(sorted_male[a].iloc[i]))

            mask_f = (sex == 1) & (age_low <= age) & (age <= age_up) & (deathstate == 0) & (np.any(diagnoses[:, diag_nums[d_id_f]], axis=1))
            mask_m = (sex == 2) & (age_low <= age) & (age <= age_up) & (deathstate == 0) & (np.any(diagnoses[:, diag_nums[d_id_m]], axis=1))

            if np.any(mask_f) and n_f > 0:
                n_f = min(n_f, np.sum(mask_f))
                rand_f = np.random.choice(np.where(mask_f)[0], size=n_f, replace=False)
                deathstate[rand_f] += year
            if np.any(mask_m) and n_m > 0:
                n_m = min(n_m, np.sum(mask_m))
                rand_m = np.random.choice(np.where(mask_m)[0], size=n_m, replace=False)
                deathstate[rand_m] += year

    return deathstate, dr_eff_female, dr_eff_male

# -------------------
# SIMULATION LOOP
# -------------------
starting_year = baseline_year
diag_count_py = []
cl_sz = []

for year in range(0, sim_years):
    for i, sex_code in enumerate([2, 1]):
        n_births = int(df_birth.loc[starting_year][i])
        age = np.append(age, [0] * n_births)
        sex = np.append(sex, [sex_code] * n_births)
        cluster = np.append(cluster, np.zeros((n_births, sim_years + 1)), axis=0)
        deathstate = np.append(deathstate, [0] * n_births)
        diagnoses = np.append(diagnoses, np.zeros((n_births, n_diagnoses)), axis=0)
        migr = np.append(migr, [0] * n_births)

    age, sex, deathstate, diagnoses, cluster, migr = migration(
        age, sex, deathstate, diagnoses, cluster,
        df_migr_female, df_migr_male, str(starting_year),
        year, age_groups_fy, sim_years, migr, starting_year
    )

    for t, (age_low, age_up) in enumerate(age_groups_fy):
        for c in range(n_clusters):
            mask_f = (sex == 1) & (age_low <= age) & (age <= age_up) & (deathstate == 0) & (cluster[:, year] == c)
            mask_m = (sex == 2) & (age_low <= age) & (age <= age_up) & (deathstate == 0) & (cluster[:, year] == c)
            cluster[mask_f, year + 1] = c
            cluster[mask_m, year + 1] = c

    age += 1

    df_d_w = df_death_numbers_w[year]
    df_d_m = df_death_numbers_m[year]
    deathstate = dying_indep(age, sex, deathstate, df_death_indep_w, df_death_indep_m, starting_year, age_groups_fy)
    deathstate = dying(age, sex, deathstate, diagnoses, df_d_w, df_d_m, starting_year, age_groups_fy, diag_index, diag_nums)[0]

    cl_sz.append([np.count_nonzero((cluster[:, year + 1] == c) & (deathstate == 0)) for c in range(n_clusters)])
    diag_count_py.append([0] * n_diagnoses)

    print(f"Finished year {starting_year}")
    starting_year += 1


Finished year 2015
Finished year 2016
Finished year 2017
Finished year 2018
Finished year 2019
Finished year 2020
Finished year 2021
Finished year 2022
Finished year 2023
Finished year 2024
